In [9]:
# violence post-processing
# violence class ID= 6
# person class ID = 1

import sys

with open('../samples/violence_sample_positive.txt', 'r') as f:
    input_file = f.read()
#with open('../samples/violence_sample_negative.txt', 'r') as f:
#    input_file = f.read()

# data format: [class, track_id, x, y, width, height]
input_data = [x.split(' ') for x in input_file.split('\n')]
input_data = sorted(input_data, key=lambda x: x[0])

res = []

violences = []
person_mid_points = []
for _bound_box in input_data:
    if _bound_box[0] == '6':
        violence_x_range = [float(_bound_box[1]), float(_bound_box[1]) + float(_bound_box[3])]
        violence_y_range = [float(_bound_box[2]), float(_bound_box[2]) + float(_bound_box[4])]
        violences.append([violence_x_range, violence_y_range])
    elif _bound_box[0] == '0':
        person_mid_points.append([float(_bound_box[1]) + float(_bound_box[3]) / 2, float(_bound_box[2]) + float(_bound_box[4]) / 2])
        res.append(_bound_box)

for violence in violences:
    person_cnt = 0
    for mid_point in person_mid_points:
        if violence[0][0] < mid_point[0] < violence[0][1] and violence[1][0] < mid_point[1] < violence[1][1]:
            person_cnt += 1
        if person_cnt >= 2:
            res.append(['6', violence[0][0], violence[1][0], violence[0][1] - violence[0][0], violence[1][1] - violence[1][0]])
            
print(res, sep='\n')
        

[['0', '140.42', '57.25', '19.49', '37.93'], ['0', '162.94', '68.3', '22.09', '39.76'], ['6', 138.77, 55.31, 47.29999999999998, 53.78999999999999]]


In [ ]:
# Dumping post-processing
# Person class ID = 0
# Trash class ID = 8
# Dumping class ID = 22

import sys